# TensorFlow / Keras 里，独热编码（One-Hot Encoding）
有多种方式可以实现，下面我给你讲解一下 官方推荐的自带方法，并区分场景。
## 1. tf.one_hot() —— 最基础的方法

TensorFlow 提供的原生函数，直接把整数类别转成独热向量。

In [ ]:
import tensorflow as tf

# 假设类别是 0,1,2
labels = [0, 1, 2]

one_hot = tf.one_hot(labels, depth=3)  
print(one_hot)

输出
tf.Tensor(
[[1. 0. 0.]   # 类别 0
 [0. 1. 0.]   # 类别 1
 [0. 0. 1.]], # 类别 2
 shape=(3, 3), dtype=float32)


### ✅ 常用参数：
 - `indices`：输入的整数序列（类别标签）。
 - `depth`：类别总数（维度大小）。
 - `on_value` / `off_value`：默认为 1/0，可自定义。
 - `dtype`：结果类型，默认 float32。

### 例子：

In [ ]:
tf.one_hot([1,2], depth=4, on_value=5, off_value=-1)
# 输出: [[-1  5 -1 -1], [-1 -1  5 -1]]

👉 适合在 模型外 手动预处理标签。

---

# 2. tf.keras.utils.to_categorical()

这是 Keras 的 高层封装，专门用于把整数标签转为独热编码。

In [ ]:
from tensorflow.keras.utils import to_categorical

labels = [0, 1, 2]
one_hot = to_categorical(labels, num_classes=3)
print(one_hot)

输出：
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


### ✅ 特点：

- `num_classes`：类别总数（可不写，会自动根据最大值+1）。

- 输入必须是整数索引。

- 返回 float32 数组。

### 👉 适合在 `分类问题` 中准备 `y_train` / `y_test`。

---

# 3. tf.keras.layers.CategoryEncoding —— 作为模型层

在 模型里直接用层实现独热编码，无需在数据预处理阶段手动转换。

In [ ]:
from tensorflow.keras.layers import CategoryEncoding, Input
from tensorflow.keras.models import Model

# 输入整数类别
inp = Input(shape=(1,), dtype="int32")

# 独热编码层
x = CategoryEncoding(num_tokens=5, output_mode="one_hot")(inp)

model = Model(inp, x)

print(model(tf.constant([[0],[3],[4]])))

输出：
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]], shape=(3, 5), dtype=float32)


### ✅ 参数：

- `num_tokens`：类别数（必须指定）。

- `output_mode`：

    - `one_hot` → 独热编码

    - `multi_hot` → 多热编码（用于多标签）

    - `count` → 统计次数

### 👉 适合放在 模型的输入层，直接处理类别特征

---

# 4. tf.keras.layers.StringLookup + CategoryEncoding

如果你的类别是字符串（比如 "`cat`", "`dog`"），需要先把字符串映射为整数，再做独热编码。
#### 👉 常用于 非数值型类别特征。

In [ ]:
from tensorflow.keras.layers import StringLookup

vocab = ["cat", "dog", "fish"]
lookup = StringLookup(vocabulary=vocab, output_mode="int")

labels = tf.constant(["dog", "cat", "fish"])
int_encoded = lookup(labels)          # -> [2, 1, 3]

one_hot_layer = CategoryEncoding(num_tokens=len(vocab)+1, output_mode="one_hot")
one_hot = one_hot_layer(int_encoded)

print(one_hot)


# 📌 总结

TensorFlow 自带独热编码主要有 3 种方式：
| 方法                 | 场景              | 特点                                          |
| ------------------ | --------------- | ------------------------------------------- |
| `tf.one_hot()`     | 低层 API，直接编码整数标签 | 灵活，适合预处理                                    |
| `to_categorical()` | 高层 API，分类标签处理   | 简单，常用于 `y`                                  |
| `CategoryEncoding` | 作为模型层           | 直接集成到网络输入层，支持 one\_hot / multi\_hot / count |

- 如果你处理的是 字符串类别 → `StringLookup` + `CategoryEncoding`
- 如果你处理的是 整数类别 → `CategoryEncoding` 或 `to_categorical`
- 如果你只想快速预处理 → `tf.one_hot`


---

# 1. 什么是 tf.data.Dataset

`tf.data.Dataset` 表示一个 元素序列（sequence of elements）。

- 每个元素可以是：

    - 一个张量（`Tensor`）

    - 或者一个张量的元组（(`features`, `label`)）

    - 甚至是嵌套结构（`dict`, `tuple` 等）。

### 👉 用一句话概括：

`Dataset` = 可迭代的数据对象，用来替代传统的 `for` 循环 / `feed_dict`。

---
# 2. 创建 Dataset 的方式
### (1) 从张量创建

In [ ]:
import tensorflow as tf

# 一维张量
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5])
for x in dataset:
    print(x.numpy())

输出：

1
2
3
4
5

如果是 特征 + 标签：

In [ ]:
features = tf.constant([[1,2],[3,4],[5,6]])
labels = tf.constant([0,1,0])
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

for x,y in dataset:
    print(x.numpy(), y.numpy())


### (2) 从 generator 创建

In [ ]:

import numpy as np

def gen():
    for i in range(5):
        yield i, i*i

dataset = tf.data.Dataset.from_generator(
    gen,
    output_types=(tf.int32, tf.int32),
    output_shapes=((),())
)



### (3) 从文件创建

TFRecord 文件

In [ ]:
dataset = tf.data.TFRecordDataset("data.tfrecord")

In [ ]:
文本文件

In [ ]:
dataset = tf.data.TextLineDataset("data.txt")

In [ ]:
多个文件

In [ ]:
files = ["file1.tfrecord", "file2.tfrecord"]
dataset = tf.data.TFRecordDataset(files)

---

# 3. 常用操作（变换算子）
### (1) map() —— 映射（预处理）

对每个元素应用函数。

In [ ]:
dataset = dataset.map(lambda x: x+1)


### (2) batch() —— 批处理

In [ ]:
dataset = dataset.batch(2)

输入 [1,2,3,4,5] → [[1,2],[3,4],[5]]

### (3) shuffle() —— 打乱顺序

In [ ]:
dataset = dataset.shuffle(buffer_size=100)

buffer_size 越大，打乱效果越好。

### (4) repeat() —— 重复数据

In [ ]:
dataset = dataset.repeat(2)   # 数据重复两次
dataset = dataset.repeat()    # 无限重复


### (5) prefetch() —— 数据预取

In [ ]:
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

👉 训练和数据准备并行，提高 GPU 利用率。

---

# 4. 综合示例（完整输入管道）

In [ ]:
# 假设 features, labels 已经准备好
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

dataset = (dataset
           .shuffle(buffer_size=1000)   # 打乱
           .map(lambda x,y: (x/255.0, y))  # 归一化
           .batch(32)                   # 批处理
           .prefetch(tf.data.AUTOTUNE)  # 预取
)

然后交给模型训练：

model.fit(dataset, epochs=10)


---

# 5. Dataset 的迭代方式

In [ ]:
for batch_x, batch_y in dataset.take(1):
    print(batch_x.shape, batch_y)


- `take`(1) 只取 1 个 `batch`。

- 如果数据量大，建议不要 `list(dataset)`，要逐批迭代。

# 6. 高级技巧

## 1.多线程 map（加速预处理）

In [ ]:
dataset = dataset.map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)


### 2.交错加载多个文件

In [ ]:
dataset = tf.data.Dataset.list_files("data/*.tfrecord")
dataset = dataset.interleave(tf.data.TFRecordDataset,
                             cycle_length=4,
                             num_parallel_calls=tf.data.AUTOTUNE)


### 3.缓存数据（避免重复计算）

In [ ]:
dataset = dataset.cache()


# 7. 总结记忆

- 创建方式：from_tensor_slices、from_generator、TFRecordDataset

- 变换算子：map（预处理）、batch（批次）、shuffle（打乱）、repeat（重复）、prefetch（加速）

### 最佳实践：
```python
dataset = (dataset
           .shuffle()
           .map(..., num_parallel_calls=AUTOTUNE)
           .batch()
           .prefetch(AUTOTUNE))

---

# 📌 tf.data.Dataset 常用方法速查表

| 方法 | 作用 | 示例 |
|------|------|------|
| `from_tensor_slices(data)` | 从张量/数组创建数据集 | `Dataset.from_tensor_slices((x, y))` |
| `from_generator(gen, output_types, output_shapes)` | 从生成器创建数据集 | `Dataset.from_generator(gen, (tf.int32, tf.int32), ((),()))` |
| `TFRecordDataset(filenames)` | 从 TFRecord 文件创建 | `Dataset.TFRecordDataset("data.tfrecord")` |
| `TextLineDataset(filenames)` | 从文本文件创建 | `Dataset.TextLineDataset("data.txt")` |
| `list_files(pattern)` | 读取多个文件路径（可用于交错加载） | `Dataset.list_files("data/*.tfrecord")` |
| `map(map_func, num_parallel_calls)` | 对每个元素应用函数（预处理） | `ds.map(lambda x: x/255.0, num_parallel_calls=AUTOTUNE)` |
| `batch(batch_size, drop_remainder=False)` | 按批次取数据 | `ds.batch(32)` |
| `shuffle(buffer_size)` | 打乱数据顺序 | `ds.shuffle(1000)` |
| `repeat(count=None)` | 重复数据集，`None` 表示无限重复 | `ds.repeat(10)` |
| `prefetch(buffer_size)` | 预取数据，加速训练 | `ds.prefetch(AUTOTUNE)` |
| `take(n)` | 只取前 n 个元素 | `ds.take(1)` |
| `cache(filename=None)` | 缓存数据到内存/文件 | `ds.cache()` |
| `skip(n)` | 跳过前 n 个元素 | `ds.skip(10)` |
| `filter(predicate)` | 过滤不满足条件的数据 | `ds.filter(lambda x: x > 0)` |
| `flat_map(map_func)` | 展开并合并多个子数据集 | `ds.flat_map(lambda x: Dataset.from_tensor_slices(x))` |
| `interleave(map_func, cycle_length)` | 交错加载多个子数据集 | `files.interleave(TFRecordDataset, cycle_length=4)` |
| `enumerate()` | 给元素加上索引 | `ds.enumerate()` |

---

✅ **推荐最佳实践组合：**
```python
dataset = (tf.data.Dataset.from_tensor_slices((features, labels))
           .shuffle(buffer_size=1000)
           .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(32)
           .prefetch(tf.data.AUTOTUNE))
